# CS5228 Predict HDB Pricing in Singapore Using MLP

This code was ran in GCP n1-highmem-4 8 vCPUs 52GiB
https://gcpinstances.doit-intl.com/?selected=n1-highmem-4

High computational time and resources required to train the MLP. 

In [53]:
%reload_ext autoreload
%autoreload 2

# Import packages
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import random
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import KFold

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# To avoid hogging the total GBs for this kernel
for gpu in tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

import sys
sys.path.append("src/")
from src.preprocessing_train_test import *
from src.preprocessing_merge import *
from src.clustering import *

# Load HDB Dataset

In [6]:
dataset = pd.read_csv("../data/backup/merge_auxiliary_data_train.csv")

Preprocess helper function

In [7]:
def preprocess_mlp(train_df, test_df):

    # Perform shuffling of training data, so that batches can be more representative
    shuffled_training_df = train_df.sample(frac=1)

    X_train_shuffled = shuffled_training_df.loc[:, ~shuffled_training_df.columns.isin(["resale_price"])]
    y_train_shuffled = shuffled_training_df["resale_price"]
    y_test = test_df['resale_price']

    # Train a min-max scaler on the train dataset, and transform on both train and test
    sc = StandardScaler()
    sc_train = sc.fit(X_train_shuffled)
    X_train_scaled = sc_train.transform(X_train_shuffled)
    X_test_scaled = sc_train.transform(test_df[X_train_shuffled.columns])

    # Scale data down by 100,000 
    y_train_scaled = y_train_shuffled/100000
    y_test_scaled = y_test/100000

    # print("X_train shape: ", X_train_scaled.shape)
    # print("y_train shape: ", y_train_scaled.shape)
    # print("X_test shape: ", X_test_scaled.shape)
    # print("y_test shape: ", y_test_scaled.shape)

    return X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled

Model training helper function

In [8]:
def mlp_model(fold, X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled, hidden_layers_dict, initializer, \
              optimizer_set="Adam", input_neurons=20, activation_hidden='relu', activation_final='linear', learning_rate=0.01, \
              epochs=200, batch_size=24): 

    tf.keras.backend.clear_session() 

    # Build model architecture
    if initializer == "RandomUniform": 
        initializer_func = tf.keras.initializers.RandomUniform(minval=0., maxval=1., seed=42)
    elif initializer == "RandomNormal":
        initializer_func = tf.keras.initializers.RandomNormal(mean=0., stddev=1., seed=42)
    elif initializer == "GlorotUniform": 
        initializer_func = tf.keras.initializers.GlorotUniform()
    elif initializer == "HeUniform":
        initializer_func = tf.keras.initializers.HeUniform()
    model = Sequential()
    model.add(Dense(input_neurons, input_dim=X_train_scaled.shape[1], activation='relu', kernel_initializer=initializer_func)) 
    for hidden_layer, hidden_neurons in hidden_layers_dict.items():
        model.add(Dense(hidden_neurons, activation=activation_hidden))
    model.add(Dense(1, activation=activation_final)) 

    # Compile model
    if optimizer_set == "Adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate) 
    elif optimizer_set == "SGD": 
        optimizer = tf.keras.optimizers.experimental.SGD(learning_rate=learning_rate, momentum=0.8) 
    elif optimizer_set == "Adagrad":
        optimizer = tf.keras.optimizers.experimental.Adagrad(learning_rate=learning_rate) 
    elif optimizer_set == "RMSprop": 
        optimizer = tf.keras.optimizers.experimental.RMSprop(learning_rate=learning_rate, momentum=0.8) 
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
    model.compile(loss='mae', optimizer=optimizer, metrics=['mae', 'mse'])

    # Fit model
    history = model.fit(X_train_scaled, y_train_scaled, epochs=epochs, batch_size=batch_size, \
                          validation_data=[X_test_scaled, y_test_scaled], callbacks=[es])
    display(history)

    # Save model
    model_name = "fold" + str(fold) + "_" + str(epochs) + "ep_" + str(batch_size) + "batches_" + str(learning_rate) \
                 + "rate_" + initializer + "_" + str(list(hidden_layers_dict.values())) + "_" + optimizer_set
    pickle.dump(model, \
              open(f'model/model_mlp/mlp_model_{model_name}.pkl', 'wb'))

    # Get predicted house price on training data
    yhat_train = 100000*model.predict(X_train_scaled)
    train_MAE = mean_absolute_error(y_train_scaled*100000, yhat_train)
    train_MSE = mean_squared_error(y_train_scaled*100000, yhat_train)

    # Get predicted house pricing on testing data
    yhat_test = 100000*model.predict(X_test_scaled)
    test_MAE = mean_absolute_error(y_test_scaled*100000, yhat_test)
    test_MSE = mean_squared_error(y_test_scaled*100000, yhat_test)

    return model, history, train_MAE, train_MSE, test_MAE, test_MSE


K-fold cross validation run

In [9]:
def kfold_crossvalidate(data, k, tuning_grid, model_path): 

    # Initialize score storing
    scores = {}

    # Define k-fold cross-validation
    kfold = KFold(n_splits=k, shuffle=True, random_state=1)

    # Loop through each fold of k-fold cross-validation
    for i, (train_index, test_index) in enumerate(kfold.split(data)):
        print(f'start fold {i}', len(train_index), len(test_index))

        # # Preprocess data for this fold
        train_df, test_df = preprocess_train_test(data.iloc[list(train_index)], data.iloc[list(test_index)])
        X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = preprocess_mlp(train_df, test_df)

        # Fit the model for this fold
        for epochs in tuning_grid["epochs"]: 
            for batch_size in tuning_grid["batch_size"]: 
                for learning_rate in tuning_grid["learning_rate"]: 
                    for initializer in tuning_grid["initializers"]:
                        for optimizer_set in tuning_grid["optimizers"]:
                            for hidden_layers_dict in tuning_grid["hidden_layers_dict"]:
                                example = (i, epochs, batch_size, learning_rate, initializer, optimizer_set, str(list(hidden_layers_dict.keys())))
                                #if len([i for i in scores.keys() if i == example])<1:
                                print("Running", example)
                                model, history, train_mae, train_mse, valid_mae, valid_mse = mlp_model(i, X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled, hidden_layers_dict, initializer, \
                                    optimizer_set=optimizer_set, input_neurons=20, activation_hidden='relu', activation_final='linear', learning_rate=learning_rate, epochs=epochs, batch_size=batch_size)
                                scores[(i, epochs, batch_size, learning_rate, initializer, optimizer_set, str(list(hidden_layers_dict.values())))] = (train_mae, valid_mae, train_mse, valid_mse)
                                with open(f"model/model_mlp/{model_path}.pkl", "wb") as f: 
                                    pickle.dump(scores, f)
                                # else: 
                                #     continue
    return scores

In [61]:
def single_run(data, param_dict, features_drop, model_path): 

    # Initialize score storing
    scores = {}

    # Data Preprocessing
    
    X_train, X_test, y_train, y_test = train_test_split(data.loc[:, ~data.columns.isin(["resale_price"])], data["resale_price"], test_size=0.3, random_state=42)
    train = pd.concat([X_train, y_train], axis=1)
    test = pd.concat([X_test, y_test], axis=1)
    
    train_df, test_df = preprocess_train_test(train, test)
    train_df = train_df.loc[:, ~train_df.columns.isin(features_drop)]
    test_df = test_df.loc[:, ~test_df.columns.isin(features_drop)]
    print(train_df.columns)
    X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = preprocess_mlp(train_df, test_df)
    
    # Model Fitting
    hidden_layers_dict = param_dict["hidden_layers_dict"]
    initializer = param_dict["initializers"]
    optimizer_set = param_dict["optimizers"]
    learning_rate = param_dict["learning_rate"]
    epochs = param_dict["epochs"]
    batch_size = param_dict["batch_size"]
    model, history, train_mae, train_mse, valid_mae, valid_mse = mlp_model(0, X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled, hidden_layers_dict, initializer, \
        optimizer_set=optimizer_set, input_neurons=20, activation_hidden='relu', activation_final='linear', learning_rate=learning_rate, epochs=epochs, batch_size=batch_size)
    
    # Save scores and models
    scores[(epochs, batch_size, learning_rate, initializer, optimizer_set, str(list(hidden_layers_dict.values())))] = (train_mae, valid_mae, train_mse, valid_mse)
    with open(f"model/model_mlp/single_model/{model_path}.pkl", "wb") as f: 
        pickle.dump(scores, f)

    return model, history, scores

## Cross validation run 

In [69]:
# Testing cross validation run
hidden_layers_dict1 = {"hidden_layer1": 40}
hidden_layers_dict2 = {"hidden_layer1": 30}
hidden_layers_dict3 = {"hidden_layer1": 50}
hidden_layers_dict4 = {"hidden_layer1": 40, "hidden_layer2": 40}
tuning_grid_testing = {"epochs": [5, 10], \
               "batch_size": [64, 128, 256], \
               "learning_rate": [0.005, 0.01], \
               "initializers": ["RandomUniform", "RandomNormal"],
              "hidden_layers_dict": [hidden_layers_dict1, hidden_layers_dict2], 
              "optimizers": ["Adam", "RMSprop"]}

kfold_scores = kfold_crossvalidate(dataset, k=2, tuning_grid=tuning_grid_testing)

start fold 0 215866 215866
['hidden_layer1']
Epoch 1/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.3766 - mae: 0.3766 - mse: 0.4076 - val_loss: 0.2172 - val_mae: 0.2172 - val_mse: 0.0906
Epoch 2/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.2108 - mae: 0.2108 - mse: 0.0882 - val_loss: 0.1989 - val_mae: 0.1989 - val_mse: 0.0780
Epoch 3/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.1939 - mae: 0.1939 - mse: 0.0749 - val_loss: 0.1959 - val_mae: 0.1959 - val_mse: 0.0751
Epoch 4/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.1863 - mae: 0.1863 - mse: 0.0693 - val_loss: 0.1857 - val_mae: 0.1857 - val_mse: 0.0661
Epoch 5/5
3371/3371 [==============================] - 15s 5ms/step - loss: 0.1808 - mae: 0.1808 - mse: 0.0655 - val_loss: 0.1777 - val_mae: 0.1777 - val_mse: 0.0630


INFO:tensorflow:Assets written to: ram://10e4e561-2b8d-4d79-aecd-042f62aec787/assets
6746/6746 [==============================] - 9s 1ms/step
['hidden_layer1']
Epoch 1/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.3911 - mae: 0.3911 - mse: 0.3718 - val_loss: 0.2294 - val_mae: 0.2294 - val_mse: 0.1016
Epoch 2/5
3371/3371 [==============================] - 15s 5ms/step - loss: 0.2134 - mae: 0.2134 - mse: 0.0903 - val_loss: 0.1943 - val_mae: 0.1943 - val_mse: 0.0739
Epoch 3/5
3371/3371 [==============================] - 13s 4ms/step - loss: 0.1942 - mae: 0.1942 - mse: 0.0751 - val_loss: 0.1828 - val_mae: 0.1828 - val_mse: 0.0669
Epoch 4/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.1857 - mae: 0.1857 - mse: 0.0702 - val_loss: 0.1864 - val_mae: 0.1864 - val_mse: 0.0717
Epoch 5/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.1798 - mae: 0.1798 - mse: 0.0653 - val_loss: 0.1806 - val_mae: 0.1806 - val_mse: 0.0681


INFO:tensorflow:Assets written to: ram://83d2bf98-e8ca-4cf3-8038-bc0a49fad770/assets
6746/6746 [==============================] - 9s 1ms/step
['hidden_layer1']
Epoch 1/5
3371/3371 [==============================] - 16s 4ms/step - loss: 0.2942 - mae: 0.2942 - mse: 0.2657 - val_loss: 0.2690 - val_mae: 0.2690 - val_mse: 0.1502
Epoch 2/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.2223 - mae: 0.2223 - mse: 0.1058 - val_loss: 0.2176 - val_mae: 0.2176 - val_mse: 0.1007
Epoch 3/5
3371/3371 [==============================] - 16s 5ms/step - loss: 0.2153 - mae: 0.2153 - mse: 0.1089 - val_loss: 0.2941 - val_mae: 0.2941 - val_mse: 0.1856
Epoch 4/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.2122 - mae: 0.2122 - mse: 0.1179 - val_loss: 0.2053 - val_mae: 0.2053 - val_mse: 0.0885
Epoch 5/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.2083 - mae: 0.2083 - mse: 0.1044 - val_loss: 0.2441 - val_mae: 0.2441 - val_mse: 0.1321


INFO:tensorflow:Assets written to: ram://fbc3bc97-7bab-4d08-8cf3-df6c34129859/assets
6746/6746 [==============================] - 9s 1ms/step
['hidden_layer1']
Epoch 1/5
3371/3371 [==============================] - 16s 4ms/step - loss: 0.2974 - mae: 0.2974 - mse: 0.2230 - val_loss: 0.2558 - val_mae: 0.2558 - val_mse: 0.1348
Epoch 2/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.2251 - mae: 0.2251 - mse: 0.1073 - val_loss: 0.2161 - val_mae: 0.2161 - val_mse: 0.0926
Epoch 3/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.2178 - mae: 0.2178 - mse: 0.0989 - val_loss: 0.2059 - val_mae: 0.2059 - val_mse: 0.0876
Epoch 4/5
3371/3371 [==============================] - 16s 5ms/step - loss: 0.2130 - mae: 0.2130 - mse: 0.1457 - val_loss: 0.2229 - val_mae: 0.2229 - val_mse: 0.0944
Epoch 5/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.2110 - mae: 0.2110 - mse: 0.1002 - val_loss: 0.1950 - val_mae: 0.1950 - val_mse: 0.0800


INFO:tensorflow:Assets written to: ram://09c08489-e1c8-4d12-be4e-80cd17ad662d/assets
6746/6746 [==============================] - 9s 1ms/step
['hidden_layer1']
Epoch 1/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.3726 - mae: 0.3726 - mse: 0.3100 - val_loss: 0.2307 - val_mae: 0.2307 - val_mse: 0.1060
Epoch 2/5
3371/3371 [==============================] - 15s 5ms/step - loss: 0.2132 - mae: 0.2132 - mse: 0.0906 - val_loss: 0.2001 - val_mae: 0.2001 - val_mse: 0.0776
Epoch 3/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.1947 - mae: 0.1947 - mse: 0.0749 - val_loss: 0.1858 - val_mae: 0.1858 - val_mse: 0.0677
Epoch 4/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.1873 - mae: 0.1873 - mse: 0.0685 - val_loss: 0.1917 - val_mae: 0.1917 - val_mse: 0.0707
Epoch 5/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.1822 - mae: 0.1822 - mse: 0.0646 - val_loss: 0.1856 - val_mae: 0.1856 - val_mse: 0.0665


...

INFO:tensorflow:Assets written to: ram://3b8b8294-a6d6-40ba-88b3-0474d6b5fe5a/assets
6746/6746 [==============================] - 9s 1ms/step
['hidden_layer1']
Epoch 1/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.3340 - mae: 0.3340 - mse: 0.2748 - val_loss: 0.2027 - val_mae: 0.2027 - val_mse: 0.0823
Epoch 2/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.2061 - mae: 0.2061 - mse: 0.0857 - val_loss: 0.2243 - val_mae: 0.2243 - val_mse: 0.0995
Epoch 3/5
3371/3371 [==============================] - 15s 4ms/step - loss: 0.1963 - mae: 0.1963 - mse: 0.0774 - val_loss: 0.1910 - val_mae: 0.1910 - val_mse: 0.0716
Epoch 4/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.1928 - mae: 0.1928 - mse: 0.0748 - val_loss: 0.2141 - val_mae: 0.2141 - val_mse: 0.0960
Epoch 5/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.1889 - mae: 0.1889 - mse: 0.0718 - val_loss: 0.1861 - val_mae: 0.1861 - val_mse: 0.0693


INFO:tensorflow:Assets written to: ram://b450bfa7-aeee-45de-965d-f3bb62f05cbc/assets
6746/6746 [==============================] - 9s 1ms/step
['hidden_layer1']
Epoch 1/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.3210 - mae: 0.3210 - mse: 0.3050 - val_loss: 0.2249 - val_mae: 0.2249 - val_mse: 0.0990
Epoch 2/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.2060 - mae: 0.2060 - mse: 0.0857 - val_loss: 0.1967 - val_mae: 0.1967 - val_mse: 0.0770
Epoch 3/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.1955 - mae: 0.1955 - mse: 0.0771 - val_loss: 0.1947 - val_mae: 0.1947 - val_mse: 0.0735
Epoch 4/5
3371/3371 [==============================] - 15s 4ms/step - loss: 0.1910 - mae: 0.1910 - mse: 0.0732 - val_loss: 0.1927 - val_mae: 0.1927 - val_mse: 0.0739
Epoch 5/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.1884 - mae: 0.1884 - mse: 0.0717 - val_loss: 0.1849 - val_mae: 0.1849 - val_mse: 0.0663


INFO:tensorflow:Assets written to: ram://11d4f53c-d93e-4e3d-8dad-13f464f19098/assets
6746/6746 [==============================] - 9s 1ms/step
['hidden_layer1']
Epoch 1/5
3371/3371 [==============================] - 16s 4ms/step - loss: 0.3171 - mae: 0.3171 - mse: 0.4588 - val_loss: 0.2659 - val_mae: 0.2659 - val_mse: 0.1402
Epoch 2/5
3371/3371 [==============================] - 16s 5ms/step - loss: 0.2750 - mae: 0.2750 - mse: 0.4778 - val_loss: 0.2672 - val_mae: 0.2672 - val_mse: 0.1407
Epoch 3/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.2622 - mae: 0.2622 - mse: 0.3308 - val_loss: 0.2364 - val_mae: 0.2364 - val_mse: 0.1124
Epoch 4/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.2601 - mae: 0.2601 - mse: 0.7741 - val_loss: 0.2497 - val_mae: 0.2497 - val_mse: 0.1248
Epoch 5/5
3371/3371 [==============================] - 14s 4ms/step - loss: 0.2635 - mae: 0.2635 - mse: 1.4161 - val_loss: 0.3130 - val_mae: 0.3130 - val_mse: 0.1612


INFO:tensorflow:Assets written to: ram://6958e4f4-1a8b-48dd-9bd8-945d58ab7732/assets
4806/6746 [====================>.........] - ETA: 2s

KeyboardInterrupt: 

In [53]:
# Full cross validation run

hidden_layers_dict1 = {"hidden_layer1": 40}
hidden_layers_dict2 = {"hidden_layer1": 30}
hidden_layers_dict3 = {"hidden_layer1": 50}
hidden_layers_dict4 = {"hidden_layer1": 40, "hidden_layer2": 40}
tuning_grid = {"epochs": [50, 100, 200], \
               "batch_size": [64, 128, 256], \
               "learning_rate": [0.0001, 0.001, 0.01], \
               "initializers": ["RandomUniform", "RandomNormal", "GlorotUniform", "HeUniform"],
              "hidden_layers_dict": [hidden_layers_dict1, hidden_layers_dict2, hidden_layers_dict3, hidden_layers_dict4], 
              "optimizers": ["Adam", "RMSprop", "SGD"]}

kfold_scores = kfold_crossvalidate(dataset, k=5, tuning_grid=tuning_grid, model_path="checkpoint_v2")

start fold 0 345385 86347
Running (0, 50, 64, 0.0001, 'RandomNormal', 'SGD', "['hidden_layer1']")
Epoch 1/50
5391/5391 [==============================] - 17s 3ms/step - loss: 0.9341 - mae: 0.9341 - mse: 1.6947 - val_loss: 0.6011 - val_mae: 0.6011 - val_mse: 0.5870
Epoch 2/50
5391/5391 [==============================] - 18s 3ms/step - loss: 0.5750 - mae: 0.5750 - mse: 0.5379 - val_loss: 0.5513 - val_mae: 0.5513 - val_mse: 0.4955
Epoch 3/50
5391/5391 [==============================] - 16s 3ms/step - loss: 0.5321 - mae: 0.5321 - mse: 0.4626 - val_loss: 0.5140 - val_mae: 0.5140 - val_mse: 0.4311
Epoch 4/50
5391/5391 [==============================] - 16s 3ms/step - loss: 0.4947 - mae: 0.4947 - mse: 0.4021 - val_loss: 0.4800 - val_mae: 0.4800 - val_mse: 0.3824
Epoch 5/50
5391/5391 [==============================] - 16s 3ms/step - loss: 0.4661 - mae: 0.4661 - mse: 0.3592 - val_loss: 0.4525 - val_mae: 0.4525 - val_mse: 0.3421
Epoch 6/50
5391/5391 [==============================] - 18s 3ms/ste

INFO:tensorflow:Assets written to: ram://86dc97dd-ae33-476a-a916-75609afc3f68/assets
2699/2699 [==============================] - 4s 1ms/step
Running (0, 50, 64, 0.0001, 'RandomNormal', 'SGD', "['hidden_layer1']")
Epoch 1/50
5391/5391 [==============================] - 19s 3ms/step - loss: 1.0103 - mae: 1.0103 - mse: 2.0631 - val_loss: 0.6146 - val_mae: 0.6146 - val_mse: 0.6267
Epoch 2/50
5391/5391 [==============================] - 16s 3ms/step - loss: 0.5896 - mae: 0.5896 - mse: 0.5757 - val_loss: 0.5670 - val_mae: 0.5670 - val_mse: 0.5307
Epoch 3/50
5391/5391 [==============================] - 16s 3ms/step - loss: 0.5527 - mae: 0.5527 - mse: 0.5037 - val_loss: 0.5478 - val_mae: 0.5478 - val_mse: 0.4913
Epoch 4/50
5391/5391 [==============================] - 16s 3ms/step - loss: 0.5243 - mae: 0.5243 - mse: 0.4518 - val_loss: 0.5100 - val_mae: 0.5100 - val_mse: 0.4244
Epoch 5/50
5391/5391 [==============================] - 18s 3ms/step - loss: 0.4956 - mae: 0.4956 - mse: 0.4031 - val_

INFO:tensorflow:Assets written to: ram://ef2737ad-71be-422e-b88d-eae99a3363ab/assets
2699/2699 [==============================] - 4s 2ms/step
Running (0, 50, 64, 0.0001, 'RandomNormal', 'SGD', "['hidden_layer1']")
Epoch 1/50
5391/5391 [==============================] - 17s 3ms/step - loss: 0.8762 - mae: 0.8762 - mse: 1.4029 - val_loss: 0.5957 - val_mae: 0.5957 - val_mse: 0.5795
Epoch 2/50
5391/5391 [==============================] - 16s 3ms/step - loss: 0.5718 - mae: 0.5718 - mse: 0.5320 - val_loss: 0.5555 - val_mae: 0.5555 - val_mse: 0.4988
Epoch 3/50
5391/5391 [==============================] - 16s 3ms/step - loss: 0.5381 - mae: 0.5381 - mse: 0.4742 - val_loss: 0.5234 - val_mae: 0.5234 - val_mse: 0.4506
Epoch 4/50
5391/5391 [==============================] - 18s 3ms/step - loss: 0.5094 - mae: 0.5094 - mse: 0.4276 - val_loss: 0.4935 - val_mae: 0.4935 - val_mse: 0.4028
Epoch 5/50
5391/5391 [==============================] - 16s 3ms/step - loss: 0.4791 - mae: 0.4791 - mse: 0.3801 - val_

INFO:tensorflow:Assets written to: ram://928954f6-7d36-43b0-b310-5fa23afe0548/assets
2699/2699 [==============================] - 4s 1ms/step
Running (0, 50, 64, 0.0001, 'RandomNormal', 'SGD', "['hidden_layer1']")
Epoch 1/50
5391/5391 [==============================] - 18s 3ms/step - loss: 0.7851 - mae: 0.7851 - mse: 1.1150 - val_loss: 0.5802 - val_mae: 0.5802 - val_mse: 0.5517
Epoch 2/50
5391/5391 [==============================] - 16s 3ms/step - loss: 0.5544 - mae: 0.5544 - mse: 0.5026 - val_loss: 0.5278 - val_mae: 0.5278 - val_mse: 0.4550
Epoch 3/50
5391/5391 [==============================] - 18s 3ms/step - loss: 0.5061 - mae: 0.5061 - mse: 0.4205 - val_loss: 0.4864 - val_mae: 0.4864 - val_mse: 0.3927
Epoch 4/50
5391/5391 [==============================] - 16s 3ms/step - loss: 0.4721 - mae: 0.4721 - mse: 0.3675 - val_loss: 0.4579 - val_mae: 0.4579 - val_mse: 0.3459
Epoch 5/50
5391/5391 [==============================] - 16s 3ms/step - loss: 0.4418 - mae: 0.4418 - mse: 0.3260 - val_

INFO:tensorflow:Assets written to: ram://17441b5f-ec77-4ea0-8bf7-8ace7f6ab0ad/assets
2699/2699 [==============================] - 4s 1ms/step
Running (0, 50, 64, 0.0001, 'RandomNormal', 'SGD', "['hidden_layer1']")
Epoch 1/50


...

INFO:tensorflow:Assets written to: ram://63239018-7005-4f79-9435-83633eb32fd8/assets
2699/2699 [==============================] - 4s 1ms/step
Running (0, 50, 64, 0.0001, 'RandomNormal', 'SGD', "['hidden_layer1']")
Epoch 1/50
5391/5391 [==============================] - 20s 3ms/step - loss: 0.2378 - mae: 0.2378 - mse: 0.1350 - val_loss: 0.1869 - val_mae: 0.1869 - val_mse: 0.0674
Epoch 2/50
5391/5391 [==============================] - 18s 3ms/step - loss: 0.1874 - mae: 0.1874 - mse: 0.0698 - val_loss: 0.1794 - val_mae: 0.1794 - val_mse: 0.0634
Epoch 3/50
5391/5391 [==============================] - 20s 4ms/step - loss: 0.1779 - mae: 0.1779 - mse: 0.0633 - val_loss: 0.1786 - val_mae: 0.1786 - val_mse: 0.0656
Epoch 4/50
5391/5391 [==============================] - 18s 3ms/step - loss: 0.1727 - mae: 0.1727 - mse: 0.0598 - val_loss: 0.1753 - val_mae: 0.1753 - val_mse: 0.0596
Epoch 5/50
5391/5391 [==============================] - 18s 3ms/step - loss: 0.1697 - mae: 0.1697 - mse: 0.0577 - val_

INFO:tensorflow:Assets written to: ram://69fa0f55-34dd-4400-8ab0-8d7ca7b49432/assets
2699/2699 [==============================] - 4s 1ms/step
Running (0, 50, 64, 0.0001, 'RandomNormal', 'SGD', "['hidden_layer1']")
Epoch 1/50
5391/5391 [==============================] - 19s 3ms/step - loss: 0.4893 - mae: 0.4893 - mse: 0.4978 - val_loss: 0.4020 - val_mae: 0.4020 - val_mse: 0.2822
Epoch 2/50
5391/5391 [==============================] - 17s 3ms/step - loss: 0.3852 - mae: 0.3852 - mse: 0.2577 - val_loss: 0.3663 - val_mae: 0.3663 - val_mse: 0.2349
Epoch 3/50
5391/5391 [==============================] - 16s 3ms/step - loss: 0.3102 - mae: 0.3102 - mse: 0.1759 - val_loss: 0.2477 - val_mae: 0.2477 - val_mse: 0.1159
Epoch 4/50
5391/5391 [==============================] - 18s 3ms/step - loss: 0.2264 - mae: 0.2264 - mse: 0.0988 - val_loss: 0.2133 - val_mae: 0.2133 - val_mse: 0.0880
Epoch 5/50
5391/5391 [==============================] - 16s 3ms/step - loss: 0.2074 - mae: 0.2074 - mse: 0.0840 - val_

INFO:tensorflow:Assets written to: ram://3e6d9bf9-5e29-47de-9301-8e87b86efe3e/assets
2699/2699 [==============================] - 3s 1ms/step
Running (0, 50, 64, 0.0001, 'RandomNormal', 'SGD', "['hidden_layer1']")
Epoch 1/50
5391/5391 [==============================] - 17s 3ms/step - loss: 0.4763 - mae: 0.4763 - mse: 0.4585 - val_loss: 0.4059 - val_mae: 0.4059 - val_mse: 0.2893
Epoch 2/50
5391/5391 [==============================] - 18s 3ms/step - loss: 0.3833 - mae: 0.3833 - mse: 0.2559 - val_loss: 0.3603 - val_mae: 0.3603 - val_mse: 0.2284
Epoch 3/50
5391/5391 [==============================] - 16s 3ms/step - loss: 0.3046 - mae: 0.3046 - mse: 0.1735 - val_loss: 0.2415 - val_mae: 0.2415 - val_mse: 0.1144
Epoch 4/50
5391/5391 [==============================] - 17s 3ms/step - loss: 0.2248 - mae: 0.2248 - mse: 0.1010 - val_loss: 0.2164 - val_mae: 0.2164 - val_mse: 0.0937
Epoch 5/50
5391/5391 [==============================] - 19s 3ms/step - loss: 0.2103 - mae: 0.2103 - mse: 0.0886 - val_

INFO:tensorflow:Assets written to: ram://3d62ca17-b839-4d42-8c6e-6d75cce316f5/assets
2699/2699 [==============================] - 4s 1ms/step
Running (0, 50, 64, 0.0001, 'RandomNormal', 'SGD', "['hidden_layer1']")
Epoch 1/50
5391/5391 [==============================] - 17s 3ms/step - loss: 0.4894 - mae: 0.4894 - mse: 0.4791 - val_loss: 0.3953 - val_mae: 0.3953 - val_mse: 0.2685
Epoch 2/50
5391/5391 [==============================] - 17s 3ms/step - loss: 0.3780 - mae: 0.3780 - mse: 0.2477 - val_loss: 0.3608 - val_mae: 0.3608 - val_mse: 0.2259
Epoch 3/50
5391/5391 [==============================] - 17s 3ms/step - loss: 0.3138 - mae: 0.3138 - mse: 0.1794 - val_loss: 0.2530 - val_mae: 0.2530 - val_mse: 0.1222
Epoch 4/50
5391/5391 [==============================] - 17s 3ms/step - loss: 0.2313 - mae: 0.2313 - mse: 0.1044 - val_loss: 0.2181 - val_mae: 0.2181 - val_mse: 0.0931
Epoch 5/50
5391/5391 [==============================] - 16s 3ms/step - loss: 0.2117 - mae: 0.2117 - mse: 0.0894 - val_

KeyboardInterrupt: 

In [ ]:
# Second cross validation run based on lesson learnt from first round run
hidden_layers_dict1 = {"hidden_layer1": 40, "hidden_layer2": 40}
hidden_layers_dict2 = {"hidden_layer1": 40, "hidden_layer2": 40, "hidden_layer3":40}
tuning_grid = {"epochs": [100, 150], \
               "batch_size": [64, 128, 256], \
               "learning_rate": [0.0001, 0.0005], \
               "initializers": ["GlorotUniform"],
              "hidden_layers_dict": [hidden_layers_dict1, hidden_layers_dict2], 
              "optimizers": ["Adam", "RMSprop"]}

kfold_scores = kfold_crossvalidate(dataset, k=5, tuning_grid=tuning_grid, model_path="checkpoint_v3")

start fold 0 345385 86347
Running (0, 50, 64, 0.0001, 'RandomNormal', 'SGD', "['hidden_layer1']")
Epoch 1/100
5391/5391 [==============================] - 19s 3ms/step - loss: 0.5697 - mae: 0.5697 - mse: 0.7578 - val_loss: 0.3909 - val_mae: 0.3909 - val_mse: 0.2651
Epoch 2/100
5391/5391 [==============================] - 17s 3ms/step - loss: 0.3466 - mae: 0.3466 - mse: 0.2151 - val_loss: 0.2945 - val_mae: 0.2945 - val_mse: 0.1598
Epoch 3/100
5391/5391 [==============================] - 17s 3ms/step - loss: 0.2603 - mae: 0.2603 - mse: 0.1266 - val_loss: 0.2358 - val_mae: 0.2358 - val_mse: 0.1039
Epoch 4/100
5391/5391 [==============================] - 19s 3ms/step - loss: 0.2217 - mae: 0.2217 - mse: 0.0931 - val_loss: 0.2115 - val_mae: 0.2115 - val_mse: 0.0850
Epoch 5/100
5391/5391 [==============================] - 17s 3ms/step - loss: 0.2048 - mae: 0.2048 - mse: 0.0805 - val_loss: 0.2007 - val_mae: 0.2007 - val_mse: 0.0764
Epoch 6/100
5391/5391 [==============================] - 17s 3

INFO:tensorflow:Assets written to: ram://9931e5b0-b305-48ed-b9da-35f50d182e85/assets
2699/2699 [==============================] - 4s 1ms/step
Running (0, 50, 64, 0.0001, 'RandomNormal', 'SGD', "['hidden_layer1']")
Epoch 1/100
5391/5391 [==============================] - 20s 4ms/step - loss: 0.5424 - mae: 0.5424 - mse: 0.6586 - val_loss: 0.3988 - val_mae: 0.3988 - val_mse: 0.2795
Epoch 2/100
5391/5391 [==============================] - 17s 3ms/step - loss: 0.3496 - mae: 0.3496 - mse: 0.2196 - val_loss: 0.2825 - val_mae: 0.2825 - val_mse: 0.1489
Epoch 3/100
5391/5391 [==============================] - 17s 3ms/step - loss: 0.2409 - mae: 0.2409 - mse: 0.1123 - val_loss: 0.2164 - val_mae: 0.2164 - val_mse: 0.0913
Epoch 4/100
5391/5391 [==============================] - 17s 3ms/step - loss: 0.2061 - mae: 0.2061 - mse: 0.0839 - val_loss: 0.1986 - val_mae: 0.1986 - val_mse: 0.0773
Epoch 5/100
5391/5391 [==============================] - 18s 3ms/step - loss: 0.1930 - mae: 0.1930 - mse: 0.0741 -

INFO:tensorflow:Assets written to: ram://f95d36ba-7cca-4d98-80f2-16df7011c49d/assets
2699/2699 [==============================] - 4s 1ms/step
Running (0, 50, 64, 0.0001, 'RandomNormal', 'SGD', "['hidden_layer1']")
Epoch 1/100
5391/5391 [==============================] - 19s 3ms/step - loss: 0.3604 - mae: 0.3604 - mse: 0.3059 - val_loss: 0.2107 - val_mae: 0.2107 - val_mse: 0.0854
Epoch 2/100
5391/5391 [==============================] - 18s 3ms/step - loss: 0.1939 - mae: 0.1939 - mse: 0.0740 - val_loss: 0.1820 - val_mae: 0.1820 - val_mse: 0.0655
Epoch 3/100
5391/5391 [==============================] - 19s 4ms/step - loss: 0.1782 - mae: 0.1782 - mse: 0.0633 - val_loss: 0.1731 - val_mae: 0.1731 - val_mse: 0.0601
Epoch 4/100
5391/5391 [==============================] - 17s 3ms/step - loss: 0.1708 - mae: 0.1708 - mse: 0.0585 - val_loss: 0.1656 - val_mae: 0.1656 - val_mse: 0.0545
Epoch 5/100
5391/5391 [==============================] - 18s 3ms/step - loss: 0.1659 - mae: 0.1659 - mse: 0.0553 -

INFO:tensorflow:Assets written to: ram://bf7b41f8-35a5-498f-9449-29f6c1ada4e8/assets
2699/2699 [==============================] - 4s 1ms/step
Running (0, 50, 64, 0.0001, 'RandomNormal', 'SGD', "['hidden_layer1']")
Epoch 1/100
5391/5391 [==============================] - 20s 4ms/step - loss: 0.3152 - mae: 0.3152 - mse: 0.2457 - val_loss: 0.1961 - val_mae: 0.1961 - val_mse: 0.0756
Epoch 2/100
5391/5391 [==============================] - 19s 3ms/step - loss: 0.1859 - mae: 0.1859 - mse: 0.0684 - val_loss: 0.1757 - val_mae: 0.1757 - val_mse: 0.0614
Epoch 3/100
5391/5391 [==============================] - 21s 4ms/step - loss: 0.1744 - mae: 0.1744 - mse: 0.0608 - val_loss: 0.1696 - val_mae: 0.1696 - val_mse: 0.0573
Epoch 4/100
5391/5391 [==============================] - 19s 3ms/step - loss: 0.1685 - mae: 0.1685 - mse: 0.0570 - val_loss: 0.1698 - val_mae: 0.1698 - val_mse: 0.0572
Epoch 5/100
5391/5391 [==============================] - 19s 3ms/step - loss: 0.1644 - mae: 0.1644 - mse: 0.0544 -

INFO:tensorflow:Assets written to: ram://a0861806-38b3-45e1-9cc7-4baeca3c0bca/assets
2699/2699 [==============================] - 4s 1ms/step
Running (0, 50, 64, 0.0001, 'RandomNormal', 'SGD', "['hidden_layer1']")
Epoch 1/100
5391/5391 [==============================] - 19s 3ms/step - loss: 0.3604 - mae: 0.3604 - mse: 0.2940 - val_loss: 0.2082 - val_mae: 0.2082 - val_mse: 0.0835
Epoch 2/100
5391/5391 [==============================] - 16s 3ms/step - loss: 0.1930 - mae: 0.1930 - mse: 0.0734 - val_loss: 0.1812 - val_mae: 0.1812 - val_mse: 0.0640
Epoch 3/100
5391/5391 [==============================] - 16s 3ms/step - loss: 0.1772 - mae: 0.1772 - mse: 0.0619 - val_loss: 0.1728 - val_mae: 0.1728 - val_mse: 0.0583
Epoch 4/100
5391/5391 [==============================] - 16s 3ms/step - loss: 0.1701 - mae: 0.1701 - mse: 0.0570 - val_loss: 0.1671 - val_mae: 0.1671 - val_mse: 0.0549
Epoch 5/100
5391/5391 [==============================] - 18s 3ms/step - loss: 0.1654 - mae: 0.1654 - mse: 0.0540 -

INFO:tensorflow:Assets written to: ram://86642595-018d-4fff-8d27-c19a83b88fdf/assets
2699/2699 [==============================] - 4s 1ms/step
Running (0, 50, 64, 0.0001, 'RandomNormal', 'SGD', "['hidden_layer1']")
Epoch 1/100
5391/5391 [==============================] - 17s 3ms/step - loss: 0.3446 - mae: 0.3446 - mse: 0.3139 - val_loss: 0.2077 - val_mae: 0.2077 - val_mse: 0.0868
Epoch 2/100
5391/5391 [==============================] - 17s 3ms/step - loss: 0.1889 - mae: 0.1889 - mse: 0.0716 - val_loss: 0.1812 - val_mae: 0.1812 - val_mse: 0.0664
Epoch 3/100
5391/5391 [==============================] - 18s 3ms/step - loss: 0.1749 - mae: 0.1749 - mse: 0.0623 - val_loss: 0.1762 - val_mae: 0.1762 - val_mse: 0.0624
Epoch 4/100
5391/5391 [==============================] - 17s 3ms/step - loss: 0.1677 - mae: 0.1677 - mse: 0.0576 - val_loss: 0.1658 - val_mae: 0.1658 - val_mse: 0.0551
Epoch 5/100
5391/5391 [==============================] - 17s 3ms/step - loss: 0.1630 - mae: 0.1630 - mse: 0.0548 -

INFO:tensorflow:Assets written to: ram://6d8e1eb5-b5b2-4629-a6ba-c552fa8ee60e/assets
2699/2699 [==============================] - 4s 1ms/step
Running (0, 50, 64, 0.0001, 'RandomNormal', 'SGD', "['hidden_layer1']")
Epoch 1/100
5391/5391 [==============================] - 19s 3ms/step - loss: 0.2402 - mae: 0.2402 - mse: 0.1353 - val_loss: 0.1911 - val_mae: 0.1911 - val_mse: 0.0733
Epoch 2/100
5391/5391 [==============================] - 19s 4ms/step - loss: 0.1760 - mae: 0.1760 - mse: 0.0615 - val_loss: 0.1643 - val_mae: 0.1643 - val_mse: 0.0546
Epoch 3/100
5391/5391 [==============================] - 18s 3ms/step - loss: 0.1674 - mae: 0.1674 - mse: 0.0558 - val_loss: 0.1710 - val_mae: 0.1710 - val_mse: 0.0583
Epoch 4/100
5391/5391 [==============================] - 18s 3ms/step - loss: 0.1626 - mae: 0.1626 - mse: 0.0527 - val_loss: 0.1600 - val_mae: 0.1600 - val_mse: 0.0521
Epoch 5/100
5391/5391 [==============================] - 20s 4ms/step - loss: 0.1598 - mae: 0.1598 - mse: 0.0508 -

INFO:tensorflow:Assets written to: ram://b1da81a0-aca4-47e1-9185-e9cbcac68293/assets
2699/2699 [==============================] - 4s 1ms/step
Running (0, 50, 64, 0.0001, 'RandomNormal', 'SGD', "['hidden_layer1']")
Epoch 1/100
5391/5391 [==============================] - 21s 4ms/step - loss: 0.2384 - mae: 0.2384 - mse: 0.1391 - val_loss: 0.1992 - val_mae: 0.1992 - val_mse: 0.0796
Epoch 2/100
5391/5391 [==============================] - 20s 4ms/step - loss: 0.1808 - mae: 0.1808 - mse: 0.0647 - val_loss: 0.1732 - val_mae: 0.1732 - val_mse: 0.0591
Epoch 3/100
5391/5391 [==============================] - 22s 4ms/step - loss: 0.1715 - mae: 0.1715 - mse: 0.0585 - val_loss: 0.1664 - val_mae: 0.1664 - val_mse: 0.0551
Epoch 4/100
5391/5391 [==============================] - 20s 4ms/step - loss: 0.1661 - mae: 0.1661 - mse: 0.0551 - val_loss: 0.1619 - val_mae: 0.1619 - val_mse: 0.0531
Epoch 5/100
5391/5391 [==============================] - 20s 4ms/step - loss: 0.1621 - mae: 0.1621 - mse: 0.0526 -

In [10]:
# Full cross validation run

hidden_layers_dict1 = {"hidden_layer1": 40, "hidden_layer2": 40}
hidden_layers_dict2 = {"hidden_layer1": 50, "hidden_layer2": 50}
tuning_grid = {"epochs": [150], \
               "batch_size": [64, 128, 256], \
               "learning_rate": [0.0001], \
               "initializers": ["GlorotUniform"],
              "hidden_layers_dict": [hidden_layers_dict1, hidden_layers_dict2], 
              "optimizers": ["RMSprop"]}

kfold_scores = kfold_crossvalidate(dataset, k=5, tuning_grid=tuning_grid, model_path="checkpoint_v4")

start fold 0 345385 86347
Running (0, 150, 64, 0.0001, 'GlorotUniform', 'RMSprop', "['hidden_layer1', 'hidden_layer2']")
Epoch 1/150


InternalError: Graph execution error:

Detected at node 'sequential/dense/MatMul' defined at (most recent call last):
    File "/opt/conda/lib/python3.7/runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "/opt/conda/lib/python3.7/runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.7/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.7/asyncio/base_events.py", line 541, in run_forever
      self._run_once()
    File "/opt/conda/lib/python3.7/asyncio/base_events.p

# Single model run

In [42]:
score = calculateTopCorrelation(dataset.select_dtypes("float"))
score

,Columns,Pearson-score
4,resale_price,1.000000
0,floor_area_sqm,0.570385
16,adult_children_ratio,0.148428
17,nearest_dist_commerical,0.128597
18,nearest_dist_market,0.121716
15,male_female_ratio,0.100189
2,longitude,0.099786
6,distance_to_mrt,0.076309
21,MinShopDist,0.074931
10,senior_citizen_count,0.070973


In [ ]:
# Third round run: Best hyperparameters, and drop features
features_drop = ['distance_to_mrt_bins_price', 'mrt_lrt_links', 'population_bins_price', \
                 'adult_children_ratio_bins_price', 'mrt_interchange_flag', \
                 'mrt_interchange_count', 'mrt_type_price']
hidden_layers_dict1 = {"hidden_layer1": 40, "hidden_layer2": 40}
param_dict = {"epochs": 100, \
              "batch_size": 64, \
              "learning_rate": 0.0001, \
              "initializers": "GlorotUniform",
              "hidden_layers_dict": hidden_layers_dict1, 
              "optimizers": "RMSprop"}
model, history, scores = single_run(dataset, param_dict, features_drop, model_path="checkpoint_v1")

In [7]:
# Generate scores dataframe to keep track of performance
scores_fold = pd.DataFrame()
for i, (key, item) in enumerate(scores.items()): 
    print(key)
    print(item)
    if i == 0: 
        scores_fold = pd.concat([pd.DataFrame(key).T, pd.DataFrame(item).T], axis=1)
    else:
        scores_fold_individual = pd.concat([pd.DataFrame(key).T, pd.DataFrame(item).T], axis=1) 
        scores_fold = pd.concat([scores_fold, scores_fold_individual], axis=0)

scores_fold.columns= ["Fold", "Epochs", "Batch_Size", "Learning_Rate", "Initializer", "Optimizer", "Hidden Layer", \
                      "Train_MAE", "Valid_MAE", "Train_MSE", "Valid_MSE"]

(0, 50, 64, 0.0001, 'RandomUniform', 'Adam', "['hidden_layer1']")
(18547.658554113936, 18693.488803854256, 667825219.0440903, 677758306.8282994)
(0, 50, 64, 0.0001, 'RandomUniform', 'Adam', "['hidden_layer1', 'hidden_layer2']")
(16966.181507471156, 17122.39417180105, 552512028.5167701, 564215702.3025129)
(0, 50, 64, 0.0001, 'RandomUniform', 'RMSprop', "['hidden_layer1']")
(15791.432109613199, 15974.525564544438, 493945210.3614243, 506886556.4817135)
(0, 50, 64, 0.0001, 'RandomUniform', 'RMSprop', "['hidden_layer1', 'hidden_layer2']")
(15061.901377688144, 15232.577054287354, 445407621.9098772, 455805285.9389062)
(0, 50, 64, 0.0001, 'RandomUniform', 'SGD', "['hidden_layer1']")
(46072.16358280178, 46258.08262648206, 3448019559.4435787, 3485492461.4705715)
(0, 50, 64, 0.0001, 'RandomUniform', 'SGD', "['hidden_layer1', 'hidden_layer2']")
(38381.15694390407, 38522.749959770554, 2589739036.3535814, 2621474025.3822665)
(0, 50, 64, 0.0001, 'RandomUniform', 'Adagrad', "['hidden_layer1']")
(13640

In [22]:
scores_fold

,Fold,Epochs,Batch_Size,Learning_Rate,Initializer,Optimizer,Hidden Layer,Train_MAE,Valid_MAE,Train_MSE,Valid_MSE
0,0,50,64,0.0001,RandomUniform,Adam,['hidden_layer1'],18547.658554,18693.488804,6.678252e+08,6.777583e+08
0,0,50,64,0.0001,RandomUniform,Adam,"['hidden_layer1', 'hidden_layer2']",16966.181507,17122.394172,5.525120e+08,5.642157e+08
0,0,50,64,0.0001,RandomUniform,RMSprop,['hidden_layer1'],15791.432110,15974.525565,4.939452e+08,5.068866e+08
0,0,50,64,0.0001,RandomUniform,RMSprop,"['hidden_layer1', 'hidden_layer2']",15061.901378,15232.577054,4.454076e+08,4.558053e+08
0,0,50,64,0.0001,RandomUniform,SGD,['hidden_layer1'],46072.163583,46258.082626,3.448020e+09,3.485492e+09
0,0,50,64,0.0001,RandomUniform,SGD,"['hidden_layer1', 'hidden_layer2']",38381.156944,38522.749960,2.589739e+09,2.621474e+09
0,0,50,64,0.0001,RandomUniform,Adagrad,['hidden_layer1'],136401.771689,135844.114041,2.917186e+10,2.850191e+10
0,0,50,64,0.0001,RandomUniform,Adagrad,"['hidden_layer1', 'hidden_layer2']",97394.659598,96829.042407,1.639160e+10,1.607883e+10
0,0,50,64,0.0001,RandomNormal,Adam,['hidden_layer1'],17944.181130,18086.234942,6.414754e+08,6.463325e+08
0,0,50,64,0.0001,RandomNormal,Adam,"['hidden_layer1', 'hidden_layer2']",17249.359255,17335.917265,5.979183e+08,6.010827e+08


In [ ]:
# pickle.dump(scores_fold, open("../results/model_mlp/checkpoint_v2.pkl", "wb"))
# pickle.dump(scores_fold, open("../results/model_mlp/checkpoint_v3.pkl", "wb"))